In [ ]:
sm = snakemake

In [ ]:
import spherpro.bro as spb
import spherpro.datastore as spd
import spherpro.library as spl
import spherpro.configuration as conf
import spherpro.db as db
import sqlalchemy as sa
import imp
import pycytools as pct
import pycytools.library
import re
import os
import pandas as pd
import numpy as np
import plotnine as gg
import spherpro.library as lib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.patches as mpatches
import pdb
import pathlib

%matplotlib notebook
%matplotlib inline


import seaborn as sns
import scipy.stats as stats
from IPython.display import display



In [ ]:
from src.config import Conf as C


In [ ]:
import spherpro.bromodules.helpers_vz as helpers_vz
imp.reload(helpers_vz)

In [ ]:
class CurVariableHelper(helpers_vz.VariableBaseHelper):
    COL_D2RIM = 'DistRim'
    COL_GENE = 'gene'
    COL_GENE_UNTAGGED = 'gene_untagged'
    COL_TAG = 'tag'
    COL_DOXO = 'doxocycline'
    COL_DILUTION = 'dilution'
    COL_GOODNAME = 'goodname'
    COL_ISNB = 'isnb'
    COL_FITTED = 'fitted'
    COL_RESID = 'residual'
    COL_IMGID = db.images.image_id.key
    COL_OBJ_NR = db.objects.object_number.key
    COL_FC = 'fc'
    COL_P = 'p'
    COL_DF = 'DF'
    COL_DELTA = 'delta'
    COL_TSTAT = 't'
    COL_TAGSTAT = 'TagStat'
    COL_POSSTAT = 'PosStat'
    COL_WORKING = 'working'
    COL_N = 'n'
    COL_N_OVEREXPR = 'n_overexpr'
    COL_FC_CENS = 'fc_cens'
    COL_NB = 'nb'
    COL_P_CORR = 'p_corrected'
    COL_ISSIG = 'is_sig_sel'
    COL_FITCONDITIONNAME = 'FitConditionName'
    COL_VALUES = db.object_measurements.value.key

    COL_COEFNAME = 'coefname'
    
    LAB_CELLLINE = 'Cellline'
    LAB_TP = 'Timepoint'
    LAB_CONC = 'Size'
    
V = CurVariableHelper

In [ ]:
V.COL_MODTYPE = 'modtype'
V.COL_MODEL = 'model'
V.COL_MODELCLASS = 'modelclass'
V.COL_METAL = 'metal'
V.COL_MARKER_CLASS = 'marker_class'
V.COL_IS_CC = 'is_cc'

V.COL_CONDITION = V.COL_CONDNAME
V.COL_TP = 'timepoint'
V.COL_CONC = 'concentration'
V.COL_CELLLINE = 'cellline'

V.COL_CHANNEL = 'channel'

In [ ]:
class C(C):
    main_conds = sm.params.main_conds
    main_conds_cl = sm.params.celllines
    all_conds =  sm.params.all_conds
    fig2_cond = [main_conds[1]]

In [ ]:
V.COL_MEASTYPE = db.measurement_types.measurement_type.key
V.COL_MEASID = db.measurements.measurement_id.key
V.COL_VALUE = db.object_measurements.value.key
MEAS_INTENSITY = 'Intensity'

In [ ]:
cmap_cells = [colors.to_hex(c) for c in sns.color_palette('deep')]

In [ ]:
cmap_cells

In [ ]:
fn_config = sm.input.fn_config
fn_modstats_out = sm.input.fn_modstats
fol_out_paper = pathlib.Path(sm.output.fol_out)


bro = spb.get_bro(fn_config)


hpr = helpers_vz.HelperVZ(bro)

dat_pannelcsv = hpr.get_pannelcsv()

dat_measmeta = hpr.get_measuremeta(dat_pannelcsv,additional_measfilt=sa.and_(db.stacks.stack_name == 'Dist',
                                                                            db.measurements.measurement_name == 'dist-rim',
                                                                            db.ref_planes.channel_name == 'object'))

dat_imgmeta = hpr.get_imgmeta()
#dat_imgmeta[V.COL_CONDLEVEL] = dat_imgmeta[V.COL_CONDID].map(str)
#dat_imgmeta[V.COL_SITELEVEL] = dat_imgmeta[V.COL_SITEID].map(str)


fil_good_meas = hpr.get_fil_good_meas(dat_measmeta)







#dat_measmeta = dat_measmeta.merge(dat_pannelcsv[[V.COL_METAL, V.COL_MARKER_CLASS]],how='left')
dat_measmeta = dat_measmeta.merge(dat_pannelcsv, how='left')
dat_measmeta[V.COL_IS_CC] = dat_measmeta[V.COL_IS_CC] == 1
#dat_measmeta[V.COL_IS_CC] = False
dat_measmeta = dat_measmeta.set_index(V.COL_MEASID, drop=False)

In [ ]:
fol_out = pathlib.Path(sm.output.fol_out)
os.makedirs(fol_out, exist_ok=True)

In [ ]:
dat_mod_stats = pd.read_csv(fn_modstats_out)

In [ ]:
dat_mod_stats

In [ ]:
chans = dat_pannelcsv.query(f'{V.COL_WORKING} == True')[V.COL_METAL]
dat_mod_stats = dat_mod_stats.query(f'{V.COL_CHANNELNAME} in {list(chans)}')

In [ ]:
chans

In [ ]:
r_var= V.COL_R2 = COL_R2VAR = 'r2_adj_corr'
COL_MODELSEQ = 'modelseq'

In [ ]:
# creats all possible combinations:
from itertools import chain, combinations
import itertools

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)  # allows duplicate elements
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [ ]:
seq= ['dist', 'int', 'nb', 'self']

In [ ]:
def all_delta_r(dat, seq):
    n_el = len(seq)
    seq_deltas = dict()
    for perm in itertools.permutations(range(n_el)):
        r_delta = np.empty(shape=n_el)
        r_cur = 0
        for i in range(n_el):
            mod = '_'.join(seq[j] for j in sorted(perm[:i+1]))
            d_cur = dat[mod]-r_cur
            r_delta[i] = d_cur
            r_cur = dat[mod]
        seq_lab = '_'.join(seq[k] for k in perm)
        seq_deltas[seq_lab] = r_delta
    return seq_deltas

In [ ]:
def calc_deltavar(d):
    dat = dict(d[[V.COL_MODELCLASS, V.COL_R2]].to_records(index=False))
    mod_var = pd.Series(all_delta_r(dat, seq)).map(np.var)
    mod_var.index.name = COL_MODELSEQ
    mod_var.name = COL_R2VAR
    return mod_var

In [ ]:
dat_r2var = dat_mod_stats.groupby([V.COL_CONDNAME, V.COL_CHANNELNAME]).apply(calc_deltavar).stack().reset_index().rename(columns={0: COL_R2VAR})

In [ ]:
dat_r2var.groupby([COL_MODELSEQ])[COL_R2VAR].mean().sort_values()

In [ ]:
def get_groupmin(dat, group_vars, val_var):
    return dat.loc[dat.groupby(group_vars)[val_var].idxmin()]

In [ ]:

get_groupmin(dat_r2var, [V.COL_CONDNAME, V.COL_CHANNELNAME], COL_R2VAR)[COL_MODELSEQ].value_counts()

In [ ]:
d = dat_r2var.groupby([COL_MODELSEQ, V.COL_CONDNAME])[COL_R2VAR].mean().reset_index()
get_groupmin(d, [V.COL_CONDNAME], COL_R2VAR)[COL_MODELSEQ].value_counts()

In [ ]:
d = dat_r2var.groupby([COL_MODELSEQ, V.COL_CHANNELNAME])[COL_R2VAR].mean().reset_index()
get_groupmin(d, [V.COL_CHANNELNAME], COL_R2VAR)[COL_MODELSEQ].value_counts()

## Additional analyses

Calculate for each order:
- how much absolute variability is additionally explained
- What fraction of additional variability is explained at each step

-> This can be used to get the additional variability explained when averaging over all orders

In [ ]:
def all_delta_rstats(dat, seq):
    n_el = len(seq)
    seq_deltas = dict()
    for perm in itertools.permutations(range(n_el)):
        r_delta = np.empty(shape=n_el)
        r_res = np.empty(shape=n_el)
        r_cur = 0
        for i in range(n_el):
            mod = '_'.join(seq[j] for j in sorted(perm[:i+1]))
            d_cur = dat[mod]-r_cur
            r_delta[i] = d_cur
            r_res[i] = 1-r_cur
            r_cur = dat[mod]
        seq_lab = '_'.join(seq[k] for k in perm)
        seq_deltas[seq_lab] = pd.DataFrame({'r_delta':r_delta,
                                            'r_res': r_res,
                                            'submod': [seq[k] for k in perm]})
    return seq_deltas

In [ ]:
def calc_deltavar(d):
    dat = dict(d[[V.COL_MODELCLASS, V.COL_R2]].to_records(index=False))
    mod_var = pd.concat(all_delta_rstats(dat, seq),names=['mod_order',None])
    return mod_var

In [ ]:
dat_r2_det = dat_mod_stats.groupby([V.COL_CONDNAME, V.COL_CHANNELNAME]).apply(calc_deltavar).reset_index()

In [ ]:
dat_r2_det.head()

In [ ]:
dat_r2_det.groupby([V.COL_CONDNAME, V.COL_CHANNELNAME, 'submod'])['r_delta'].mean().groupby(['submod']).mean()

In [ ]:
dat_r2_det.groupby([V.COL_CONDNAME, V.COL_CHANNELNAME, 'submod']).apply(lambda x: np.mean(x['r_delta']/x['r_res'])).groupby(['submod']).mean()

In [ ]:
dat_r2_det.groupby([V.COL_CONDNAME, V.COL_CHANNELNAME, 'submod']).apply(lambda x: np.mean(x['r_delta']/x['r_res'])).groupby(['submod']).mean()

In [ ]:
dat_r2_det

In [ ]:
dat_r2_det.query('mod_order=="dist_nb_self_int"').groupby(['submod']).mean()

In [ ]:
dat_r2avg = (dat_r2_det.groupby([V.COL_CONDNAME, V.COL_CHANNELNAME, 'submod'])
 .apply(lambda x: pd.Series({'r2_average': np.mean(x['r_delta']), 'r2_average_ofresid': np.mean(x['r_delta']/x['r_res'])}))
 #.rename('r2_average')
 .reset_index()
)

In [ ]:
dat_r2avg

In [ ]:
dat_r2avg.query('channel_name == "Yb171"')

In [ ]:
from pandas.api.types import CategoricalDtype

In [ ]:
V.COL_SUBMOD = 'submod'

In [ ]:
dat_layout = pd.read_csv(bro.data.conf['layout_csv']['path'])


dat_layout=dat_layout.rename(columns={bro.data.conf['layout_csv']['condition_col']: V.COL_CONDNAME})

dat_condmeta=(dat_layout.groupby(by=[V.COL_CELLLINE, V.COL_CONC, V.COL_TP])['well'].count()
   .reset_index()
   .assign(**{V.COL_CONDNAME: 
              lambda x: x.apply(
                  lambda g: f'{g[V.COL_CELLLINE]}_c{g[V.COL_CONC]}_tp{g[V.COL_TP]}', axis=1)}))

dat_condmeta[V.COL_CELLLINE] = pd.Categorical(dat_condmeta[V.COL_CELLLINE], categories=C.main_conds_cl)

In [ ]:
dsubmod = CategoricalDtype(categories=reversed(['dist', 'nb', 'self', 'int']))

In [ ]:
dat_r2avg[V.COL_SUBMOD ] = dat_r2avg[V.COL_SUBMOD ].astype(dsubmod) 

In [ ]:
dat_r2avg[V.COL_SUBMOD ]

In [ ]:
(dat_r2avg
 .merge(dat_condmeta)
 .groupby(by=[V.COL_CELLLINE,V.COL_SUBMOD])['r2_average']
 .mean()
 .reset_index()
 .sort_values(V.COL_SUBMOD)
 >> 
gg.ggplot(gg.aes(x=V.COL_CELLLINE, y='r2_average', fill=f"pd.Categorical({V.COL_SUBMOD}, categories=reversed(['dist', 'nb', 'self', 'int']))"))
     + gg.geom_bar(stat='identity')
     +gg.coord_cartesian(ylim=(0,0.59))
    )

In [ ]:
(dat_r2avg
 .merge(dat_condmeta)
 .groupby(by=[V.COL_CELLLINE,V.COL_SUBMOD])['r2_average']
 .mean()
 .reset_index()
 .sort_values(V.COL_SUBMOD)
 >> 
gg.ggplot(gg.aes(x=V.COL_CELLLINE, y='r2_average', fill=f"pd.Categorical({V.COL_SUBMOD}, categories=reversed(['dist', 'nb', 'self', 'int']))"))
     + gg.geom_bar(stat='identity')
     +gg.coord_cartesian(ylim=(0,0.59))
    )

In [ ]:
(dat_r2avg
 .merge(dat_condmeta)
 .groupby(by=[V.COL_CELLLINE,V.COL_SUBMOD])['r2_average']
 .mean()
 .reset_index()
)

In [ ]:
(dat_r2avg
 .merge(dat_condmeta)
 .groupby(by=[V.COL_CELLLINE,V.COL_SUBMOD])['r2_average_ofresid']
 .mean()
 .reset_index()
 .sort_values(V.COL_SUBMOD)
 >> 
gg.ggplot(gg.aes(x=V.COL_CELLLINE, y='r2_average_ofresid', fill=f"pd.Categorical({V.COL_SUBMOD}, categories=reversed(['dist', 'nb', 'self', 'int']))"))
 +gg.geom_bar(gg.aes(y=1), fill='grey', stat='identity')    
 + gg.geom_bar(stat='identity')
     +gg.coord_cartesian(ylim=(0,0.59))
    )

In [ ]:
dat_condmeta

In [ ]:
dat_r2avg

In [ ]:
f'{V.COL_CELLLINE}~{V.COL_CHANNEL}+{V.COL_TP}'

In [ ]:
(dat_r2avg
 .merge(dat_condmeta)
 >> 
gg.ggplot(gg.aes(x=f'pd.Categorical({V.COL_CONC})',
                 y='r2_average',
                 fill=f"pd.Categorical({V.COL_SUBMOD}, categories=reversed(['dist', 'nb', 'self', 'int']))"))
 +gg.facet_grid(f'{V.COL_CELLLINE}~{V.COL_CHANNELNAME}+{V.COL_TP}')
 +gg.geom_bar(gg.aes(y=1), fill='grey', stat='identity')    
 + gg.geom_bar(stat='identity')
 
     +gg.coord_cartesian(ylim=(0,1))
 +gg.theme(figure_size=(35,8))
 + gg.ggtitle('')
    )

In [ ]:
(dat_r2_det
 .query('mod_order=="dist_nb_self_int"')
 .merge(dat_condmeta)
 >> 
gg.ggplot(gg.aes(x=f'pd.Categorical({V.COL_CONC})',
                 y='r_delta',
                 fill=f"pd.Categorical({V.COL_SUBMOD}, categories=reversed(['dist', 'nb', 'self', 'int']))"))
 +gg.facet_grid(f'{V.COL_CELLLINE}~{V.COL_CHANNELNAME}+{V.COL_TP}')
 +gg.geom_bar(gg.aes(y=1), fill='grey', stat='identity')    
 + gg.geom_bar(stat='identity')
     +gg.coord_cartesian(ylim=(0,1))
 +gg.theme(figure_size=(35,8))
 + gg.ggtitle('')
    )

In [ ]:
(pd.concat([dat_r2_det,
           dat_r2avg
        .rename(columns={'r2_average': 'r_delta'})
           .assign(**{'mod_order': 'average'})])
 .query(f'{V.COL_CONDNAME} == {C.fig2_cond}')
 .query(f'{V.COL_CHANNELNAME} == "Yb171"')
 #.query('mod_order=="dist_nb_self_int"')
 .merge(dat_condmeta)
 >> 
gg.ggplot(gg.aes(x=f'mod_order',
                 y='r_delta',
                 fill=f"pd.Categorical({V.COL_SUBMOD}, categories=reversed(['dist', 'nb', 'self', 'int']))"))
 +gg.facet_grid(f'{V.COL_CHANNELNAME}~.')
 +gg.geom_bar(gg.aes(y=1), fill='grey', stat='identity')    
 + gg.geom_bar(stat='identity')
     +gg.coord_cartesian(ylim=(0,1))
 +gg.theme(figure_size=(12,5))
 + gg.theme(axis_text_x = gg.element_text(angle = 90))
 + gg.ggtitle('')
    )

In [ ]:
(pd.concat([dat_r2_det,
           dat_r2avg
        .rename(columns={'r2_average': 'r_delta'})
           .assign(**{'mod_order': 'average'})])
 .query(f'{V.COL_CONDNAME} =={C.fig2_cond}')
 #.query(f'{V.COL_CHANNELNAME} == "Nd150"')
 #.query('mod_order=="dist_nb_self_int"')
 .merge(dat_condmeta)
 >> 
gg.ggplot(gg.aes(x=f'mod_order',
                 y='r_delta',
                 fill=f"pd.Categorical({V.COL_SUBMOD}, categories=reversed(['dist', 'nb', 'self', 'int']))"))
 +gg.facet_grid(f'{V.COL_CHANNELNAME}~.')
 +gg.geom_bar(gg.aes(y=1), fill='grey', stat='identity')    
 + gg.geom_bar(stat='identity')
     +gg.coord_cartesian(ylim=(0,1))
 +gg.theme(figure_size=(10,30))
 + gg.theme(axis_text_x = gg.element_text(angle = 90))
 + gg.ggtitle('')
    )

In [ ]:
(pd.concat([dat_r2_det,
           dat_r2avg
        .rename(columns={'r2_average': 'r_delta'})
           .assign(**{'mod_order': 'average'})])
 .query(f'{V.COL_CONDNAME} =={C.fig2_cond}')
 #.query(f'{V.COL_CHANNELNAME} == "Nd150"')
 #.query('mod_order=="dist_nb_self_int"')
 .merge(dat_condmeta)
 >> 
gg.ggplot(gg.aes(x=f'mod_order',
                 y='r_delta',
                 fill=f"pd.Categorical({V.COL_SUBMOD}, categories=reversed(['dist', 'nb', 'self', 'int']))"))
 +gg.facet_grid(f'{V.COL_CHANNELNAME}~.',scales='free_y')
# +gg.geom_bar(gg.aes(y=1), fill='grey', stat='identity')    
 + gg.geom_bar(stat='identity')
   #  +gg.coord_cartesian(ylim=(0,1))
 +gg.theme(figure_size=(10,30))
 + gg.theme(axis_text_x = gg.element_text(angle = 90))
 + gg.ggtitle('')
    )

In [ ]:
C.main_conds[0]

In [ ]:
(pd.concat([dat_r2_det,
           dat_r2avg
        .rename(columns={'r2_average': 'r_delta'})
           .assign(**{'mod_order': 'average'})])
 .query(f'{V.COL_CONDNAME} == "{C.main_conds[0]}"')
 .query(f'{V.COL_CHANNELNAME} == "Yb171"')
 #.query('mod_order=="dist_nb_self_int"')
 .merge(dat_condmeta)
 >> 
gg.ggplot(gg.aes(x=f'mod_order',
                 y='r_delta',
                 fill=f"pd.Categorical({V.COL_SUBMOD}, categories=reversed(['dist', 'nb', 'self', 'int']))"))
 +gg.facet_grid(f'{V.COL_CHANNELNAME}~.')
 +gg.geom_bar(gg.aes(y=1), fill='grey', stat='identity')    
 + gg.geom_bar(stat='identity')
     +gg.coord_cartesian(ylim=(0,1))
 +gg.theme(figure_size=(12,5))
 + gg.theme(axis_text_x = gg.element_text(angle = 90))
 + gg.ggtitle('')
    )

In [ ]:
(pd.concat([dat_r2_det,
           dat_r2avg
        .rename(columns={'r2_average': 'r_delta'})
           .assign(**{'mod_order': 'average'})])
 .query(f'{V.COL_CONDNAME} =={C.fig2_cond}')
 .query(f'{V.COL_CHANNELNAME} == "Yb171"')
 #.query('mod_order=="dist_nb_self_int"')
 .merge(dat_condmeta)
 >> 
gg.ggplot(gg.aes(x=f'mod_order',
                 y='r_delta',
                 fill=f"pd.Categorical({V.COL_SUBMOD}, categories=reversed(['dist', 'nb', 'self', 'int']))"))
 +gg.facet_grid(f'{V.COL_CHANNELNAME}~.')
 +gg.geom_bar(gg.aes(y=1), fill='grey', stat='identity')    
 + gg.geom_bar(stat='identity')
     +gg.coord_cartesian(ylim=(0,1))
 +gg.theme(figure_size=(12,5))
 + gg.theme(axis_text_x = gg.element_text(angle = 90))
 + gg.ggtitle('')
    )

In [ ]:
(pd.concat([dat_r2_det,
           dat_r2avg
        .rename(columns={'r2_average': 'r_delta'})
           .assign(**{'mod_order': 'average'})])
 #.query(f'{V.COL_CONDNAME} =="HT29_c1.0_te%_tp96"')
 .groupby(by=[V.COL_SUBMOD, 'mod_order'])['r_delta'].mean().reset_index()
 #.query('mod_order=="dist_nb_self_int"')
 #.merge(dat_condmeta)
 >> 
gg.ggplot(gg.aes(x=f'mod_order',
                 y='r_delta',
                 fill=f"pd.Categorical({V.COL_SUBMOD}, categories=reversed(['dist', 'nb', 'self', 'int']))"))
# +gg.facet_grid(f'{V.COL_CHANNELNAME}~.')
 +gg.geom_bar(gg.aes(y=1), fill='grey', stat='identity')    
 + gg.geom_bar(stat='identity')
     +gg.coord_cartesian(ylim=(0,0.5))
 +gg.theme(figure_size=(12,5))
 + gg.theme(axis_text_x = gg.element_text(angle = 90))
 + gg.ggtitle('')
    )

In [ ]:
d = (pd.concat([dat_r2_det,
           dat_r2avg
        .rename(columns={'r2_average': 'r_delta'})
           .assign(**{'mod_order': 'average'})])
 #.query(f'{V.COL_CONDNAME} =="HT29_c1.0_te%_tp96"')
 .groupby(by=[V.COL_SUBMOD, 'mod_order'])['r_delta'].mean().reset_index()
 #.query('mod_order=="dist_nb_self_int"')
 #.merge(dat_condmeta)
    ) 
(gg.ggplot(d, gg.aes(x=f'mod_order',
                 y='r_delta',
                 fill=f"pd.Categorical({V.COL_SUBMOD}, categories=reversed(['dist', 'nb', 'self', 'int']))"))
# +gg.facet_grid(f'{V.COL_CHANNELNAME}~.')
# +gg.geom_bar(gg.aes(y=1), fill='grey', stat='identity')    
# + gg.geom_bar(stat='identity')
 +gg.geom_point()
 + gg.geom_hline(gg.aes(yintercept=d['r_delta'].mean()))
 +gg.stat_summary(gg.aes(group='mod_order'),fun_data = 'mean_se', geom = "errorbar")
     +gg.coord_cartesian(ylim=(0,0.5))

 +gg.theme(figure_size=(12,5))
 + gg.theme(axis_text_x = gg.element_text(angle = 90))
 + gg.ggtitle('')
    )

In [ ]:
(pd.concat([dat_r2_det,
           dat_r2avg
        .rename(columns={'r2_average': 'r_delta'})
           .assign(**{'mod_order': 'average'})])
 .query(f'{V.COL_CONDNAME} =={C.fig2_cond}')
 .query(f'{V.COL_CHANNELNAME} == "Yb171"')
 #.query('mod_order=="dist_nb_self_int"')
 .merge(dat_condmeta)
 >> 
gg.ggplot(gg.aes(x=f'mod_order',
                 y='r_delta',
                 fill=f"pd.Categorical({V.COL_SUBMOD}, categories=reversed(['dist', 'nb', 'self', 'int']))"
                ))
 +gg.facet_grid(f'{V.COL_CHANNELNAME}~.')
 +gg.geom_bar(gg.aes(y=1), fill='grey', stat='identity', position='identity')    
 + gg.geom_bar(stat='identity')
  + gg.geom_hline(gg.aes(yintercept=d['r_delta'].mean()))
# +gg.stat_summary(gg.aes(group='mod_order'),fun_data = 'mean_se', geom = "point")
  +gg.stat_summary(gg.aes(group='mod_order'),fun_data = 'mean_se', geom = "errorbar")
     +gg.coord_cartesian(ylim=(0,0.5))

     +gg.coord_cartesian(ylim=(0,1))
 +gg.theme(figure_size=(12,5))
 + gg.theme(axis_text_x = gg.element_text(angle = 90))
 + gg.guides(fill=gg.guide_legend(title='Submodel'))
 + gg.ggtitle(' ')
 + gg.xlab('Model fitting order')
 + gg.ylab('Additional variability explained')
    )

In [ ]:
def get_pord(x):
    var = x['mod_order']
    if var == 'average':
        var = 'dist_nb_self_int'
    return str(np.where(np.flip(np.array(var.split('_'))) == x['submod'])[0][0])

p = (pd.concat([dat_r2_det,
           dat_r2avg
        .rename(columns={'r2_average': 'r_delta'})
           .assign(**{'mod_order': 'average'})])
 .query(f'{V.COL_CONDNAME} =={C.fig2_cond}')
 .query(f'{V.COL_CHANNELNAME} == "Yb171"')
 #.query('mod_order=="dist_nb_self_int"')
 .merge(dat_condmeta)
 .assign(p_ord = lambda d: d.apply(get_pord ,axis=1))
 >> 
gg.ggplot(gg.aes(x=f'mod_order',
                 y='r_delta',
                 group='p_ord',
                 fill=f"pd.Categorical({V.COL_SUBMOD}, categories=reversed(['dist', 'nb', 'self', 'int']))"
                ))
 +gg.facet_grid(f'{V.COL_CHANNELNAME}~.')
 +gg.geom_bar(gg.aes(y=1), fill='grey', stat='identity', position='identity')    
 + gg.geom_bar(stat='identity')
     +gg.coord_cartesian(ylim=(0,1))
 +gg.theme(figure_size=(7,2))
 + gg.theme(axis_text_x = gg.element_text(angle = 90),
           text=gg.element_text(size=6))
 + gg.guides(fill=gg.guide_legend(title='Submodel'))
 + gg.ggtitle('Barplots of all possible permutations\n'
             'of model order\n'
             'for pS6 in the HT29 example condition')
 + gg.xlab('Model fitting order')
 + gg.ylab('Additional variability explained')
    )
p
gg.ggsave(p, fol_out / 'all_permutations.pdf')

In [ ]:


(dat_r2_det
     .assign(p_ord = lambda d: d.apply(get_pord ,axis=1))
)

In [ ]:


(dat_r2_det
     .assign(p_ord = lambda d: d.apply(get_pord ,axis=1))
)

In [ ]:
p = (pd.concat([dat_r2_det,
           dat_r2avg
        .rename(columns={'r2_average': 'r_delta'})
           .assign(**{'mod_order': 'average'})])
 .query(f'{V.COL_CONDNAME} =={C.fig2_cond}')
 .query(f'{V.COL_CHANNELNAME} == "Yb171"')
 #.query('mod_order=="dist_nb_self_int"')
 .merge(dat_condmeta)
 >> 
gg.ggplot(gg.aes(x=f'mod_order',
                 y='r_delta',
                 fill=f"pd.Categorical({V.COL_SUBMOD}, categories=reversed(['dist', 'nb', 'self', 'int']))"
                ))
 +gg.facet_grid(f'{V.COL_CHANNELNAME}~.')
 +gg.geom_bar(gg.aes(y=1), fill='grey', stat='identity', position='identity')    
 + gg.geom_bar(stat='identity')
     +gg.coord_cartesian(ylim=(0,1))
 +gg.theme(figure_size=(7.5,1))
 + gg.theme(axis_text_x = gg.element_text(angle = 90, size=6),
           axis_title_x = gg.element_text( size=6),
           axis_title_y = gg.element_text( size=6),
            axis_text_y = gg.element_text( size=6)
           )
 + gg.guides(fill=gg.guide_legend(title='Submodel'))
 + gg.ggtitle(' ')
 + gg.xlab('Model fitting order')
 + gg.ylab('Additional variability explained')
    )

In [ ]:
p

In [ ]:
gg.ggsave(p, fol_out / 'all_orders_example.pdf')

In [ ]:
dat_r2_det

In [ ]:
dat_r2var

In [ ]:
min_mean = dat_r2var.groupby(COL_MODELSEQ).mean().min()
        
(dat_r2var >>
    gg.ggplot(gg.aes(x=COL_MODELSEQ,
                 y=COL_R2VAR))
# +gg.facet_grid(f'{V.COL_CHANNELNAME}~.')
# +gg.geom_bar(gg.aes(y=1), fill='grey', stat='identity')    
# + gg.geom_bar(stat='identity')
 +gg.geom_violin()
    # +gg.coord_cartesian(ylim=(0,0.5))
 + gg.geom_hline(yintercept=min_mean, color='gray')
 +gg.stat_summary(gg.aes(group=COL_MODELSEQ),fun_data = 'mean_se', geom = "errorbar")
  +gg.stat_summary(gg.aes(group=COL_MODELSEQ),fun_data = 'mean_se', geom = "point")
 +gg.theme(figure_size=(12,2))
 + gg.theme(axis_text_x = gg.element_text(angle = 90))
 + gg.ggtitle('')
 + gg.xlab('Model fitting order')
 + gg.ylab('Variance of submodel contributions')
    #  +gg.coord_cartesian(ylim=(0,0.3))
    )

In [ ]:
dat_r2_det

In [ ]:
d = (pd.concat([dat_r2_det,
           dat_r2avg
        .rename(columns={'r2_average': 'r_delta'})
           .assign(**{'mod_order': 'average'})])
       .rename(columns={'mod_order': COL_MODELSEQ})
 #.query(f'{V.COL_CONDNAME} =="HT29_c1.0_te%_tp96"')
 .groupby(by=[V.COL_CONDNAME, V.COL_CHANNELNAME, COL_MODELSEQ])['r_delta'].var().reset_index())

In [ ]:
dat_r2var.merge(d)

In [ ]:
min_mean = d.groupby(COL_MODELSEQ).mean().min()

p = (d >>
    gg.ggplot(gg.aes(x=COL_MODELSEQ,
                 y='r_delta'))
# +gg.facet_grid(f'{V.COL_CHANNELNAME}~.')
# +gg.geom_bar(gg.aes(y=1), fill='grey', stat='identity')    
# + gg.geom_bar(stat='identity')
 +gg.geom_violin()
    # +gg.coord_cartesian(ylim=(0,0.5))
 + gg.geom_hline(yintercept=min_mean, color='gray')
 +gg.stat_summary(gg.aes(group=COL_MODELSEQ),fun_data = 'mean_se', geom = "errorbar")
  +gg.stat_summary(gg.aes(group=COL_MODELSEQ),fun_data = 'mean_se', geom = "point")
 +gg.theme(figure_size=(7.5,1.))
 + gg.theme(axis_text_x = gg.element_text(angle = 90, size=6),
           axis_title_x = gg.element_text( size=6),
           axis_title_y = gg.element_text( size=6),
            axis_text_y = gg.element_text( size=6)
           )
 + gg.ggtitle('')
 + gg.xlab('Model fitting order')
 + gg.ylab('Variance of submodel contributions')
    #  +gg.coord_cartesian(ylim=(0,0.3))
    )
p

In [ ]:
gg.ggsave(p, fol_out / 'variance_of_order_all.pdf')

In [ ]:
d['norm_var'] = (d.groupby(by=[V.COL_CONDNAME, V.COL_CHANNELNAME])['r_delta']
      .transform(lambda x: x-x.min())
      
)
d

In [ ]:
#min_mean = d2.groupby(COL_MODELSEQ).mean().min()

( d >>
    gg.ggplot(gg.aes(x=COL_MODELSEQ,
                 y='norm_var'))
# +gg.facet_grid(f'{V.COL_CHANNELNAME}~.')
# +gg.geom_bar(gg.aes(y=1), fill='grey', stat='identity')    
# + gg.geom_bar(stat='identity')
 #+gg.geom_violin()
    # +gg.coord_cartesian(ylim=(0,0.5))
 #+ gg.geom_hline(yintercept=min_mean, color='gray')
# +gg.stat_summary(gg.aes(group=COL_MODELSEQ),fun_data = 'mean_se', geom = "errorbar")
  +gg.stat_summary(gg.aes(group=COL_MODELSEQ),fun_data = 'mean_se', geom = "point")
 +gg.theme(figure_size=(12,3))
 + gg.theme(axis_text_x = gg.element_text(angle = 90))
 + gg.ggtitle('')
 + gg.xlab('Model fitting order')
 + gg.ylab('Variance of submodel contributions-minimal variance per condition/marker')
  +gg.coord_cartesian(ylim=(0,0.05))
    )

-> Like this it would be even clearer that either dist_nb_self_int and co usually have the lowest variance

In [ ]:
gg.ggsave(p, fol_out / 'all_orders_example.pdf')

In [ ]:
dat_r2_det

In [ ]:
dat_r2var

In [ ]:
min_mean = dat_r2var.groupby(COL_MODELSEQ).mean().min()
        
(dat_r2var >>
    gg.ggplot(gg.aes(x=COL_MODELSEQ,
                 y=COL_R2VAR))
# +gg.facet_grid(f'{V.COL_CHANNELNAME}~.')
# +gg.geom_bar(gg.aes(y=1), fill='grey', stat='identity')    
# + gg.geom_bar(stat='identity')
 +gg.geom_violin()
    # +gg.coord_cartesian(ylim=(0,0.5))
 + gg.geom_hline(yintercept=min_mean, color='gray')
 +gg.stat_summary(gg.aes(group=COL_MODELSEQ),fun_data = 'mean_se', geom = "errorbar")
  +gg.stat_summary(gg.aes(group=COL_MODELSEQ),fun_data = 'mean_se', geom = "point")
 +gg.theme(figure_size=(12,2))
 + gg.theme(axis_text_x = gg.element_text(angle = 90))
 + gg.ggtitle('')
 + gg.xlab('Model fitting order')
 + gg.ylab('Variance of submodel contributions')
    #  +gg.coord_cartesian(ylim=(0,0.3))
    )

In [ ]:
dat_r2_det

In [ ]:
d = (pd.concat([dat_r2_det,
           dat_r2avg
        .rename(columns={'r2_average': 'r_delta'})
           .assign(**{'mod_order': 'average'})])
       .rename(columns={'mod_order': COL_MODELSEQ})
 #.query(f'{V.COL_CONDNAME} =="HT29_c1.0_te%_tp96"')
 .groupby(by=[V.COL_CONDNAME, V.COL_CHANNELNAME, COL_MODELSEQ])['r_delta'].var().reset_index())

In [ ]:
dat_r2var.merge(d)

In [ ]:
min_mean = d.groupby(COL_MODELSEQ).mean().min()

p = (d >>
    gg.ggplot(gg.aes(x=COL_MODELSEQ,
                 y='r_delta'))
# +gg.facet_grid(f'{V.COL_CHANNELNAME}~.')
# +gg.geom_bar(gg.aes(y=1), fill='grey', stat='identity')    
# + gg.geom_bar(stat='identity')
 +gg.geom_violin()
    # +gg.coord_cartesian(ylim=(0,0.5))
 + gg.geom_hline(yintercept=min_mean, color='gray')
 +gg.stat_summary(gg.aes(group=COL_MODELSEQ),fun_data = 'mean_se', geom = "errorbar")
  +gg.stat_summary(gg.aes(group=COL_MODELSEQ),fun_data = 'mean_se', geom = "point")
 +gg.theme(figure_size=(7.5,1.))
 + gg.theme(axis_text_x = gg.element_text(angle = 90, size=6),
           axis_title_x = gg.element_text( size=6),
           axis_title_y = gg.element_text( size=6),
            axis_text_y = gg.element_text( size=6)
           )
 + gg.ggtitle('')
 + gg.xlab('Model fitting order')
 + gg.ylab('Variance of submodel contributions')
    #  +gg.coord_cartesian(ylim=(0,0.3))
    )
p

In [ ]:
gg.ggsave(p, fol_out / 'variance_of_order_all.pdf')

In [ ]:
d['norm_var'] = (d.groupby(by=[V.COL_CONDNAME, V.COL_CHANNELNAME])['r_delta']
      .transform(lambda x: x-x.min())
      
)
d

In [ ]:
#min_mean = d2.groupby(COL_MODELSEQ).mean().min()

( d >>
    gg.ggplot(gg.aes(x=COL_MODELSEQ,
                 y='norm_var'))
# +gg.facet_grid(f'{V.COL_CHANNELNAME}~.')
# +gg.geom_bar(gg.aes(y=1), fill='grey', stat='identity')    
# + gg.geom_bar(stat='identity')
 #+gg.geom_violin()
    # +gg.coord_cartesian(ylim=(0,0.5))
 #+ gg.geom_hline(yintercept=min_mean, color='gray')
# +gg.stat_summary(gg.aes(group=COL_MODELSEQ),fun_data = 'mean_se', geom = "errorbar")
  +gg.stat_summary(gg.aes(group=COL_MODELSEQ),fun_data = 'mean_se', geom = "point")
 +gg.theme(figure_size=(12,3))
 + gg.theme(axis_text_x = gg.element_text(angle = 90))
 + gg.ggtitle('')
 + gg.xlab('Model fitting order')
 + gg.ylab('Variance of submodel contributions-minimal variance per condition/marker')
  +gg.coord_cartesian(ylim=(0,0.05))
    )

-> Like this it would be even clearer that either dist_nb_self_int and co usually have the lowest variance